In [ ]:
import re
import pandas as pd


data = {
    'City': ['Samara', '', 'Volgograd', 'Khabarovsk', 'Mechta', '' , 'Borki', 'Nazran', 'Nagornyy', 'Barvikha', 'Groznyy'],
    'Province': [
        "Samarskaya Oblast'", 'North Ossetia', "Volgogradskaya Oblast'",
        "Khabarovskiy Kray", "Bryanskaya Oblast'", "Chechenskaya Respublika", 'Leningradskaya Oblast',
        "Respublika Ingushetiya", 'Komi', 'Moskovskaya Oblast', 'Stavropol'
    ],
    'municipality': [
        "Чернореченская улица, 67, Самара, 443013",
        "Государственный природный заказник Цейский, Алагирский район, Республика Северная Осетия — Алания",
        "Черноморская улица, 42, Волгоград, 400012",
        "парк Динамо, Хабаровск",
        "Игрицкое сельское поселение, Комаричский район, Брянская область",
        "село Бердыкель, городской округ Аргун, Чеченская Республика",
        "41К-452, Ленинградская область",
        "Кавказская улица, 17, Назрань, Республика Ингушетия",
        "муниципальное образование Усть-Кулом, муниципальный район Усть-Куломский, Республика Коми",
        "территория Барвиха-2, 45, деревня Рождественно, Одинцовский городской округ, Московская область",
        "Новоселицкий муниципальный округ, Ставропольский край"
    ]
}

df = pd.DataFrame(data)

def extract_admin_unit(address, city, region):
    if not isinstance(address, str):
        address = str(address)

    patterns = [
        r'(муниципальный район\s+[А-Яа-яёЁ\s-]+)', 
        r'(городской округ\s+[А-Яа-яёЁ\s-]+)',        # например: «городской округ Аргун»
        r'([А-Яа-яёЁ\s-]+городской округ)', 
        r'([А-Яа-яёЁ\s-]+муниципальный округ)',       # например: «Новоселицкий муниципальный округ»
        r'([А-Яа-яёЁ\s-]+городской район)',           # например: «Комаричский городской район»
        r'([А-Яа-яёЁ\s-]+район)'                      # например: «Алагирский район»
    ]

    for pat in patterns:
        match = re.search(pat, address, re.IGNORECASE)
        if match:
            return match.group(1).strip()  
    if city:
        return city
    return region

df['municipality'] = df.apply(
    lambda row: extract_admin_unit(row['municipality'],
                                   row['City'],
                                   row['Province']),
    axis=1
)

print(df[['City', 'Province', 'municipality']])





          City                 Province                        municipality
0       Samara       Samarskaya Oblast'                              Samara
1                         North Ossetia                    Алагирский район
2    Volgograd   Volgogradskaya Oblast'                           Volgograd
3   Khabarovsk        Khabarovskiy Kray                          Khabarovsk
4       Mechta       Bryanskaya Oblast'                   Комаричский район
5               Chechenskaya Respublika               городской округ Аргун
6        Borki    Leningradskaya Oblast                               Borki
7       Nazran   Respublika Ingushetiya                              Nazran
8     Nagornyy                     Komi  муниципальный район Усть-Куломский
9     Barvikha       Moskovskaya Oblast         Одинцовский городской округ
10     Groznyy                Stavropol    Новоселицкий муниципальный округ


In [ ]:
import re
import pandas as pd

data = {
    'District': ['', '', '', '', '', '' , '', '', '', '', '', '', 'Groznyy rayon'],
    'City': ['Samara', '', 'Volgograd', 'Khabarovsk', 'Mechta', '' , 'Borki', 'Nazran', 'Nagornyy', 'Barvikha', 'Groznyy','', ''],
    'Province': [
        "Samarskaya Oblast'", 'North Ossetia', "Volgogradskaya Oblast'",
        "Khabarovskiy Kray", "Bryanskaya Oblast'", "Chechenskaya Respublika", 'Leningradskaya Oblast',
        "Respublika Ingushetiya", 'Komi', 'Moskovskaya Oblast', 'Stavropol', 'Respublika Buryatiya', ''
    ],
    'municipality': [
        "Чернореченская улица, 67, Самара, 443013",
        "Государственный природный заказник Цейский, Алагирский район, Республика Северная Осетия — Алания",
        "Черноморская улица, 42, Волгоград, 400012",
        "парк Динамо, Хабаровск",
        "Игрицкое сельское поселение, Комаричский район, Брянская область",
        "село Бердыкель, городской округ Аргун, Чеченская Республика",
        "41К-452, Ленинградская область",
        "Кавказская улица, 17, Назрань, Республика Ингушетия",
        "муниципальное образование Усть-Кулом, муниципальный район Усть-Куломский, Республика Коми",
        "территория Барвиха-2, 45, деревня Рождественно, Одинцовский городской округ, Московская область",
        "Новоселицкий муниципальный округ, Ставропольский край",
        "сельское поселение Витимканское, сельское поселение Багдаринское, Баунтовский район, Республика Бурятия",
        ''
    ]
}

df = pd.DataFrame(data)

def extract_admin_unit(address, district, city, region):
    if not isinstance(address, str):
        address = str(address)

    # Шаблоны по приоритету:
    # (1) "муниципальный район Усть-Куломский" --> выдаём "Усть-Куломский"
    # (2) "городской округ Аргун" --> "Аргун"
    # (3) "Новоселицкий городской округ" --> "Новоселицкий"
    # (4) "Новоселицкий муниципальный округ" --> "Новоселицкий"
    # (5) "Комаричский городской район" --> "Комаричский"
    # (6) "Алагирский район" --> "Алагирский"

    patterns = [
        
        # "муниципальный район Усть-Куломский"
        r'муниципальный район\s+([А-Яа-яёЁ\s-]+)',
        r'([А-Яа-яёЁ\s-]+)\s+район',
        # "городской округ Аргун"
        r'городской округ\s+([А-Яа-яёЁ\s-]+)',
        # сельское поселение Улус-Кертское 
        r'сельское поселение\s+([А-Яа-яёЁ\s-]+)',
        r'кожуун\s+([А-Яа-яёЁ\s-]+)',
        # "Новоселицкий городской округ"
        r'([А-Яа-яёЁ\s-]+)\s+городской округ',
        # "Новоселицкий муниципальный округ"
        r'([А-Яа-яёЁ\s-]+)\s+муниципальный округ',
        # "Комаричский городской район"
        r'([А-Яа-яёЁ\s-]+)\s+городской район',
        r'([А-Яа-яёЁ\s-]+)\s+кожуун'
    ]


    for pat in patterns:
        match = re.search(pat, address, re.IGNORECASE)
        if match:
            return match.group(1).strip()

    if isinstance(district, str) and district.strip():
        clean_district = re.sub(r'\s+район$', '', district, flags=re.IGNORECASE).strip()
        return clean_district

    if isinstance(city, str) and city.strip():
        return city.strip()

    if isinstance(region, str) and region.strip():
        return region.strip()

    return None  


df['municipality'] = df.apply(
    lambda row: extract_admin_unit(row['municipality'], row['District'], row['City'], row['Province']),
    axis=1
)

print(df[['District', 'City', 'Province', 'municipality']])




         District        City                 Province    municipality
0                      Samara       Samarskaya Oblast'          Samara
1                                        North Ossetia      Алагирский
2                   Volgograd   Volgogradskaya Oblast'       Volgograd
3                  Khabarovsk        Khabarovskiy Kray      Khabarovsk
4                      Mechta       Bryanskaya Oblast'     Комаричский
5                              Chechenskaya Respublika           Аргун
6                       Borki    Leningradskaya Oblast           Borki
7                      Nazran   Respublika Ingushetiya          Nazran
8                    Nagornyy                     Komi  Усть-Куломский
9                    Barvikha       Moskovskaya Oblast     Одинцовский
10                    Groznyy                Stavropol    Новоселицкий
11                                Respublika Buryatiya     Баунтовский
12  Groznyy rayon                                        Groznyy rayon


In [ ]:
    
import re
import pandas as pd

def extract_admin_unit(address, district, city, region):
    if not isinstance(address, str):
        address = str(address)

    patterns = [
        r'муниципальный район\s+([А-Яа-яёЁ\s-]+)',
        
        r'([А-Яа-яёЁ\s-]+)\s+район',
        r'городской округ\s+([А-Яа-яёЁ\s-]+)',
        r'сельское поселение\s+([А-Яа-яёЁ\s-]+)',
        r'кожуун\s+([А-Яа-яёЁ\s-]+)',
        r'([А-Яа-яёЁ\s-]+)\s+городской округ',
        r'([А-Яа-яёЁ\s-]+)\s+муниципальный округ',
        r'([А-Яа-яёЁ\s-]+)\s+городской район',
        r'([А-Яа-яёЁ\s-]+)\s+кожуун'
    ]




    for pat in patterns:
        match = re.search(pat, address, re.IGNORECASE)
        if match:
            return match.group(1).strip()

    if isinstance(district, str):
        clean_district = re.sub(r'\s+район$', '', district, flags=re.IGNORECASE).strip()
        return clean_district

    elif isinstance(city, str):
        return city.strip()

    elif isinstance(region, str):
        return region.strip()

    return None  

input_file = "fin1yandex_рус.csv"
output_file = "события.csv"

df = pd.read_csv(input_file)

df['municipality'] = df.apply(
    lambda row: extract_admin_unit(row['municipality'], row['District'], row['City'], row['Province']),
    axis=1
)

df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Файл сохранен: {output_file}")

Файл сохранен: события.csv
